# Grid snapping

GDS represents points as integers where each point corresponds to a database unit (DBU). Typically, in most foundries, the default DBU is set to 1 nanometer (1nm).

gdsfactory snaps ports to the grid by default to avoid grid snapping errors which are common in layout designs.


![](https://i.imgur.com/SUXBWed.png)

Example of grid snapping errors.

![](https://i.imgur.com/suiHyqM.png)



In [ ]:
import gdsfactory as gf

nm = 1e-3

wg1 = gf.Component('polygon_snapping')
p1 = wg1.add_polygon([(0, 0), (1.4*nm, 0), (1.4*nm, 1*nm), (0, 1*nm)], layer=(1, 0)) # rounds to 1 nm

p2 = wg1.add_polygon([(0, 0), (1.6*nm, 0), (1.6*nm, 1*nm), (0, 1*nm)], layer=(1, 0)) # rounds to 2 nm
p2.movey(-2*nm)

p3 = wg1.add_polygon([(0, 0), (1*nm, 0), (1*nm, 1*nm), (0, 1*nm)], layer=(1, 0))
p3.movey(-4*nm)
wg1.plot()

In [ ]:
import gdsfactory as gf

nm = 1e-3

c = gf.Component('polygon_snapping_and_moving')
p1 = c.add_polygon([(0, 0), (1*nm, 0), (1*nm, 1*nm), (0, 1*nm)], layer=(1, 0)) # 1nm long
p2 = c.add_polygon([(0, 0), (1.5*nm, 0), (1.5*nm, 1*nm), (0, 1*nm)], layer=(1, 0)) # rounds to 2 nm long
p2.movex(1*nm) 
c.plot()


In [ ]:
import gdsfactory as gf

nm = 1e-3

c = gf.Component('polygon_snapping_and_moving2')
p1 = c.add_polygon([(0, 0), (1.5*nm, 0), (1.5*nm, 1*nm), (0, 1*nm)], layer=(1, 0)) # rounds to 2 nm long
p2 = c.add_polygon([(0, 0), (1*nm, 0), (1*nm, 1*nm), (0, 1*nm)], layer=(1, 0)) # 1nm long
p2.movex(1.5*nm) # rounds to 2nm movement
c.plot()


## Why you need to round gaps and lengths by 2nm?

If you center your polygon at zero, you may need to round your lengths by 2nm

In [ ]:
import gdsfactory as gf

nm = 1e-3

c = gf.Component('gap')
p1 = c.add_polygon([(-1.5*nm, 0), (1.5*nm, 0), (1.5*nm, 1*nm), (-1.5*nm, 1*nm)], layer=(1, 0)) # You wanted 3nm long but it rounds to 4 nm long
p2 = c.add_polygon([(-1*nm, 0), (1*nm, 0), (1*nm, 1*nm), (-1*nm, 1*nm)], layer=(1, 0)) # 2nm long as expected
p2.movey(-2*nm)
c.plot()


## ⚠️ Warning: Manhattan Orientations
It's crucial to always maintain ports with Manhattan orientations (0, 90, 180, 270 degrees). Non-Manhattan ports can lead to ports snapping off the grid, resulting in 1nm gaps in your layouts, which can be detrimental to the design's integrity.

Although **gdsfactory** provides functions to connect and route component ports that are off-grid or have non-Manhattan orientations, this feature is **disabled by default** for safety reasons.

> **Note:** If you intend to create off-grid ports and non-Manhattan connections, you must enable this feature manually. This should be done with caution and a thorough understanding of the potential implications on your design.

In [ ]:
c = gf.Component('non_manhattan_error_overlap')
w1 = c << gf.c.straight(length=2*nm, width=2*nm)
w1.rotate(45)

w2 = c << gf.c.straight(length=2*nm, width=2*nm)
w2.connect('o1', w1['o2'])
c # in this case ports overlap by an extra nm because of the rotation, instead of connecting them with no overlap

In [ ]:
c = gf.Component('non_manhattan_error_gap')
w1 = c << gf.components.straight(length=2*nm, width=2*nm)
w1.rotate(30)

w2 = c << gf.components.straight(length=2*nm, width=2*nm)
w2.connect('o1', w1['o2'])
c # in this case ports have a 1nm gap error because of the rotation



## Fix Non manhattan connections

The GDS format often has issues with non-manhattan shapes, due to the rounding of vertices to a unit grid and to downstream tools (i.e. DRC) which often tend to assume cell references only have rotations at 90 degree intervals. For example:

In [ ]:
import gdsfactory as gf
from gdsfactory.decorators import has_valid_transformations

gf.config.enable_offgrid_ports()  # enable off grid ports

c = gf.Component('non_manhattan')
w1 = c << gf.components.straight(length=2*nm, width=2*nm)
w1.rotate(30)

w2 = c << gf.components.straight(length=2*nm, width=2*nm)
w2.connect('o1', w1['o2'])
c = c.flatten_offgrid_references()
c

In [ ]:
import gdsfactory as gf
from gdsfactory.decorators import has_valid_transformations

@gf.cell
def demo_non_manhattan():
    c = gf.Component("bend")
    b = c << gf.components.bend_circular(angle=30)
    s = c << gf.components.straight(length=5)
    s.connect("o1", b.ports["o2"])
    return c


c1 = demo_non_manhattan()
print(has_valid_transformations(c1))
c1

if you zoom in between the bends you will see a notch between waveguides due to non-manhattan connection between the bends.

![gap](https://i.imgur.com/jBEwy9T.png)

You an fix it with the `flatten_offgrid_references` flag when you call `Component.write_gds()`.

In [ ]:
help(c1.write_gds)

In [ ]:
gdspath = c1.write_gds(flatten_offgrid_references=True)
c2 = gf.import_gds(gdspath)
has_valid_transformations(c1)  # has gap issues

In [ ]:
has_valid_transformations(c2)  # works perfect

In [ ]:
c2.plot()

If you zoom in the connection the decorator you can see it fixed the issue in `c` that we fixed in `c2` thanks to the `flatten_offgrid_references` flag.

![no gap](https://i.imgur.com/VbSgIjP.png)

### Local flattening references.

Sometimes this global `flatten_offgrid_references` can change your vias or other components by 1nm.

You can also create a copy of the cell after applying `flatten_references`. This can be applied to a single cell.


In [ ]:
c3 = c1.flatten_offgrid_references()
c3.plot()

### Default PDK `GdsWriteSettings`
If you are frequently (or even sometimes!) creating geometries like this, with non-manhattan ports and/or references with non-90-degree rotations, I highly recommend that you set `flatten_offgrid_references=True` in your PDK's `GdsWriteSettings`. If you are the PDK author, you can do this in the definition of the pdk. Or, you can modify the PDK at runtime like.

In [ ]:
pdk = gf.get_active_pdk()
pdk.gds_write_settings.flatten_offgrid_references = True

With this flag set, invalid references will be flattened by default, preventing gaps and errors in downstream tools which may not support cell references with arbitrary rotation, without needing to specify this on each GDS write.

You should note, however, that this will *not* fix all gaps between faces of unequal length, as it is *impossible* to guarantee this for diagonal line segments of unequal lengths constrained to end on integer grid values.

## Avoid Non manhattan connections

Here are some strategies to avoid or fix non-manhattan connections.

### Extrude at the end (ideal)

Some connections are hard to fix due to the ports of the bends being slightly off-center.

For that the best is to concatenate the path first and then extrude last.


In [ ]:
@gf.cell
def demo_non_manhattan_extrude_fix():
    c = gf.Component("bend")
    p1 = gf.path.arc(angle=30)
    p2 = gf.path.straight(length=5)
    p = p1 + p2
    c = p.extrude(cross_section="xs_sc")
    return c


c1 = demo_non_manhattan_extrude_fix()
c1.plot()

### Fix polygons

You can also fix polygons by merge

In [ ]:
import gdsfactory as gf

c = gf.Component("bend")
b = c << gf.components.bend_circular(angle=30)
s = c << gf.components.straight(length=5)
s.connect("o1", b.ports["o2"])
p = c.get_polygons(as_shapely_merged=True)
c2 = gf.Component("bend_fixed")
c2.add_polygon(p, layer=(1, 0))
c2.plot()

In [ ]:
import gdsfactory as gf

c = gf.Component("bend")
b = c << gf.components.bend_circular(angle=30)
s = c << gf.components.straight(length=5)
s.connect("o1", b.ports["o2"])
p = c.get_polygons(as_shapely_merged=True)
c2 = gf.Component("bend_fixed")
c2.add_polygon(p, layer=(1, 0))
c2.plot()

In [ ]:
@gf.cell
def demo_non_manhattan_merge_polygons():
    c = gf.Component("bend")
    b = c << gf.components.bend_circular(angle=30)
    s = c << gf.components.straight(length=5)
    s.connect("o1", b.ports["o2"])
    p = c.get_polygons(as_shapely_merged=True)

    c2 = gf.Component()
    c2.add_polygon(p, layer=(1, 0))
    c2.add_port("o1", port=b["o1"])
    c2.add_port("o2", port=s["o2"])
    return c2


c1 = demo_non_manhattan_merge_polygons()
c1.plot()